In [1]:
import glob
import os
import warnings

import pandas as pd
from gplately.tools import plate_isotherm_depth

if 1 == 1:
    from lib.check_files import (
        check_plate_model,
        check_source_data,
    )
    from lib.coregister_combined_point_data import run_coregister_combined_point_data
    from lib.coregister_crustal_thickness import run_coregister_crustal_thickness
    from lib.coregister_magnetic import coregister_magnetic
    from lib.erodep import calculate_erodep
    from lib.misc import (
        calculate_slab_flux,
        calculate_water_thickness,
    )
    from lib.pu import generate_grid_points
    from lib.slab_dip import calculate_slab_dip

    # Suppress occasional joblib warnings
    %env PYTHONWARNINGS=ignore::UserWarning

    warnings.simplefilter("ignore", UserWarning)


env: PYTHONWARNINGS=ignore::UserWarning


In [2]:
n_jobs = int(os.environ.get("N_JOBS", 8))
times = range(171)
verbose = False

grid_resolution = 0.5  # degrees


In [3]:
model_dir = "plate_model"
check_plate_model(model_dir, verbose=verbose)

data_dir = "source_data"
check_source_data(data_dir, verbose=verbose)
agegrid_dir = os.path.join(data_dir, "AgeGrids")
sedthick_dir = os.path.join(data_dir, "SedimentThickness")
carbonate_dir = os.path.join(data_dir, "CarbonateThickness")
co2_dir = os.path.join(data_dir, "CrustalCO2")
crustal_thickness_dir = os.path.join(data_dir, "CrustalThickness")
mag_anomaly_filename = os.path.join(
    data_dir,
    "MagneticAnomaly",
    "emag2_upcont_interpolated.nc",
)
subducted_quantities_dir = os.path.join(
    data_dir,
    "SubductedQuantities",
)
erodep_dir = os.path.join(data_dir, "ErosionDeposition")

output_dir = "extracted_data"
os.makedirs(output_dir, exist_ok=True)
subduction_data_filename = os.path.join(output_dir, "subducting_plate_data.csv")
study_area_dir = os.path.join(output_dir, "study_area_polygons")
grid_filename = None
coregistered_filename = None
output_filename = os.path.join(output_dir, "grid_data.csv")

feature_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.gpml",
    )
)
rotation_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.rot",
    )
)


### Generate grid points

In [4]:
grid_data = generate_grid_points(
    times=times,
    resolution=grid_resolution,
    polygons_dir=study_area_dir,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    n_jobs=n_jobs,
    verbose=verbose,
)


In [5]:
if grid_filename is not None:
    grid_data.to_csv(grid_filename, index=False)


### Load subduction data

In [6]:
subduction_data = pd.read_csv(subduction_data_filename)
subduction_data["plate_thickness (m)"] = plate_isotherm_depth(
    subduction_data["seafloor_age (Ma)"],
    maxiter=100,
)
subduction_data = calculate_water_thickness(subduction_data)
subduction_data = calculate_slab_flux(subduction_data)
subduction_data = calculate_slab_dip(subduction_data)


### Assign subduction data to grid

In [7]:
coregistered_data = run_coregister_combined_point_data(
    point_data=grid_data,
    subduction_data=subduction_data,
    n_jobs=n_jobs,
    verbose=verbose,
    output_filename=coregistered_filename,
)


### Assign magnetic anomaly and crustal thickness data to grid

In [8]:
coregistered_data = run_coregister_crustal_thickness(
    point_data=coregistered_data,
    input_dir=crustal_thickness_dir,
    n_jobs=n_jobs,
    verbose=verbose,
)
coregistered_data = coregister_magnetic(
    data=coregistered_data,
    filename=mag_anomaly_filename,
    n_jobs=n_jobs,
)


### Calculate cumulative erosion

In [9]:
coregistered_data = calculate_erodep(
    coregistered_data,
    input_dir=erodep_dir,
    n_jobs=n_jobs,
    column_name="erosion (m)",
    verbose=verbose,
)


banana
(3643359, 52)
(3643359,)
0 (12335, 52)
1 (12656, 52)
extract erodep TIME: 1
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/ErosionDeposition/erodep45Ma.nc', 'source_data/ErosionDeposition/er

POINT:ERODEP [-202.05516197  -40.1164566   -34.57647015 ...           nan  -69.94891846
  -73.40590645]
4 (13598, 52)
extract erodep TIME: 4
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/ErosionDe

POINT:ERODEP [ -28.89932261 -416.90811169           nan ...  -98.93800749 -498.00749161
 -495.50910574]
7 (14211, 52)
extract erodep TIME: 7
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/ErosionDe

POINT:ERODEP [          nan -215.57415029  -90.20918575 ...  -40.9365096   -40.9365096
  -33.93977112]
10 (14410, 52)
extract erodep TIME: 10
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/ErosionD

POINT:ERODEP [           nan  -765.78410726            nan ...  -522.72885464
 -1088.67993368  -629.79099573]
13 (15911, 52)
extract erodep TIME: 13
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/E

...

POINT:ERODEP [-2336.14367525  -863.37069534  -575.97084098 ... -2154.35422454
 -2762.42931088            nan]
157 (36966, 52)
extract erodep TIME: 157
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data

POINT:ERODEP [-1869.80917127 -1827.64793513            nan ...  -537.97672593
 -2456.05711795  -676.36187969]
160 (36970, 52)
extract erodep TIME: 160
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data

POINT:ERODEP [-6850.50417855 -3274.88591835 -4946.11010214 ...            nan
            nan -1204.09769049]
163 (32642, 52)
extract erodep TIME: 163
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data

POINT:ERODEP [-6993.35487384 -1102.0991431  -4829.06061898 ... -2803.95164545
 -7384.7073554  -2009.294609  ]
166 (33365, 52)
extract erodep TIME: 166
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data

POINT:ERODEP [ -609.81118409 -1205.09818191 -1204.42817723 ...  -880.33795983
  -612.50827933  -625.91620183]
169 (33473, 52)
extract erodep TIME: 169
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data

### Save to file

In [10]:
coregistered_data.to_csv(output_filename, index=False)
